In [1]:
import numpy as np
import pandas as pd
import os
import requests
import re
from bs4 import BeautifulSoup
from time import sleep
import time
from random import randint
import random
import selenium 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException


ModuleNotFoundError: No module named 'selenium'

In [ ]:
!pip install selenium

In [ ]:
hdr = {"Accept-Language": "tr-TR,tr;q=0.5",'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'}

soups = []

try:
    options = webdriver.FirefoxOptions()
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument("--enable-javascript") 
    driver = webdriver.Firefox(options=options)
    driver.implicitly_wait(0.5)
    driver.maximize_window()
    
    url = "https://www.imdb.com/search/title/?title_type=tv_series&release_date=1998-01-01,&sort=moviemeter,asc&countries=TR&count=250"
    print("1")
    driver.get(url)
    # Sayfanın dilini kontrol etmek için title etiketini al
    title_element = driver.find_element(By.TAG_NAME, 'title')
    page_language = title_element.get_attribute('lang')

# Eğer sayfa dil ayarı Türkçe değilse, dil ayarlarını değiştir
    if page_language.lower() != 'tr':
        driver.execute_script("document.documentElement.lang = 'tr';")
    
    print("2")
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'button.ipc-btn--single-padding:nth-child(1) > svg:nth-child(2)')))
    print("3")

    for _ in range(6):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        print("4")
        wait = WebDriverWait(driver, 30)
        print("5")
        try:
            print("6")
            more_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.ipc-btn--single-padding:nth-child(1) > svg:nth-child(2)')))
            driver.execute_script("arguments[0].scrollIntoView(true);", more_button)
            time.sleep(5) 
            more_button.click()
            time.sleep(30)
            print("10")
        except NoSuchElementException as nse:
            print(f"Düğme bulunamadı veya tıklanamadı. Hata: {nse}")

    
    page_source = driver.page_source
    soups = [BeautifulSoup(page_source, "html.parser")]
    print("11")


except Exception as e:
    print(f"Hata oluştu: {e}")

finally:
    if 'driver' in locals():
        driver.quit() 

In [ ]:
df = pd.DataFrame(columns=["Name", "Genre", "Date", "Actors", "AvgTime", "Season", "Episode", "Rating", "ProductionCompany", "Directors", "Writers"])

skip_count = 1000  # İlk 461 seriyi atla
current_count = 0  # Şu anda kaçıncı seri üzerindesi

for soup in soups:
    series = soup.find_all("li", {"ipc-metadata-list-summary-item"})

    for serie in series:
        current_count += 1

        if current_count <= skip_count:
            continue  # İlk 461 seriyi atla
        try:
            # Names & Links
            header = serie.find("div", {"class": "ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-43986a27-9 gaoUku dli-title"})
            a = header.find("a")
            name = (a.text).split('.')[1]
            link = "https://www.imdb.com" + a.get("href")

            # Genres
            response1 = requests.get(link, headers=hdr, timeout=20)
            response1.raise_for_status()
            soup_detail = BeautifulSoup(response1.text, "html.parser")
            genre_elements = soup_detail.find_all("span", {"class": "ipc-chip__text"})
            genres_list = [genre.text.strip() for genre in genre_elements if "back to top" not in genre.text.lower()]
            genre = ', '.join(genres_list) if genres_list else None
            
            
            #date
            date_element = serie.find("span", {"sc-43986a27-8 jHYIIK dli-title-metadata-item"})
            date = date_element.text.strip() if date_element else None
            
            
            #actors
            actors_elements = soup_detail.find_all("a", {"sc-bfec09a1-1 gCQkeh"})
            actors_list = [actor.text.strip() for actor in actors_elements[:4]]
            
            #avgTime
            avgtime_elements = soup_detail.find("ul", {"class": "ipc-inline-list ipc-inline-list--show-dividers sc-d8941411-2 cdJsTz baseAlt"})
            li_elements = avgtime_elements.find_all("li") if avgtime_elements else None
            avgTime = li_elements[-1].text.strip() if li_elements else Nonec
            
            
            #season
            season_elements = soup_detail.find("select", {"class": "ipc-simple-select__input"})
            opt_elements = season_elements.find_all("option") if season_elements else None
            season = opt_elements[1].text.strip() if opt_elements else 'none'

            
            
            #rating
            rating_elements = soup_detail.find("div", {"class": "sc-bde20123-2 cdQqzc"})
            rt_elements = rating_elements.find("span") if rating_elements else None
            rating = rt_elements.text.strip() if rt_elements else None
            
            
            #episodes
            episode_elements = soup_detail.find("h3", {"class": "ipc-title__text"})
            ep_elements = episode_elements.find("span", {"class": "ipc-title__subtext"}) if episode_elements else []
            episode = ep_elements.text.strip() if ep_elements else None
            #print(episode)
            
            
            #productionCompany
            pc_elements = soup_detail.find("a", {"class": "ipc-metadata-list-item__label ipc-metadata-list-item__label--link"}, string=lambda text: text and ("Production company" in text or "Production companies" in text))

            if pc_elements:
                ul_elements = pc_elements.find_next("div", class_="ipc-metadata-list-item__content-container").find("ul")

                if ul_elements:
                    li_list = ul_elements.find_all("li")
                    production_company = [li.text.strip() for li in li_list]

                
                    if len(production_company) == 1:
                        production_company = production_company[0]

                    
                    
            
            linkCast=soup_detail.find("a",{"class" : "ipc-link ipc-link--baseAlt ipc-link--inherit-color"}, string="Cast & crew")
            if linkCast:
                link2 = "https://www.imdb.com" + linkCast.get("href")

                try:
                    response2 = requests.get(link2, headers=hdr, timeout=20)
                    response2.raise_for_status()

                    soup_cast = BeautifulSoup(response2.text, "html.parser")

                    # (Geri kalan Cast & Crew işlemleri)

                except requests.exceptions.RequestException as e:
                    print(f"Hata oluştu: {e}")
                    continue
            else:
                print("Uyarı: 'Cast & crew' linki bulunamadı.")
                continue
            
            
            
            try:
                response2 = requests.get(link2, headers=hdr, timeout=20)
                response2.raise_for_status()

                soup_cast = BeautifulSoup(response2.text, "html.parser")
                
                
               #Directors
                directors_table = soup_cast.find('table', class_='simpleCreditsTable')
                if directors_table:
                    
                    director_names = [name.get_text(strip=True) for name in directors_table.select('.name a')[:2]]
                else:
                    None

               
                writers_tables = soup_cast.find_all('table', class_='simpleCreditsTable')
                
                if len(writers_tables) > 1:
                    writer_names = [name.get_text(strip=True) for name in writers_tables[1].select('.name a')[:2]]
                else:
                    None
            
            
            except requests.exceptions.RequestException as e:
                print(f"Hata oluştu: {e}")
                continue
                
            df = pd.concat([df, pd.DataFrame({
                "Name": [name],
                "Genre": [genre],
                "Date": [date],
                "Actors": [actors_list],
                "AvgTime": [avgTime],
                "Season": [season],
                "Episode": [episode],
                "Rating": [rating],
                "ProductionCompany": [production_company],
                "Directors": [director_names],
                "Writers": [writer_names]
            })], ignore_index=True)
            
            
            print(f"{name} {genre} {date} {', '.join(actors_list)} {avgTime} {season} {episode} {rating} {production_company} {', '.join(director_names)} {', '.join(writer_names)}")
            print("\n")
            
            time.sleep(2)
        except requests.exceptions.RequestException as e:
            print(f"Hata oluştu: {e}")
            df = pd.concat([df, pd.DataFrame({
                "Name": ["yok"],
                "Genre": ["yok"],
                "Date": ["yok"],
                "Actors": ["yok"],
                "AvgTime": ["yok"],
                "Season": ["yok"],
                "Episode": ["yok"],
                "Rating": ["yok"],
                "ProductionCompany": ["yok"],
                "Directors": ["yok"],
                "Writers": ["yok"]
            })], ignore_index=True)
            continue 

In [ ]:
df.to_csv("ikinci750.csv")